In [1]:
from config import set_environment
set_environment()

import os

from pprint import pprint

def pretty_print_docs(docs):
  print(
    f"\n{'-' * 100}\n".join(
      [f"Document {i+1}:\n\n" +
        d.page_content for i, d in enumerate(docs)]
  )
)# pprint(response)


# James Briggs


## [Better Llama 2 with Retrieval Augmented Generation (RAG)](https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb)
`RAG`, `LLAMA2`

## Chatbots with RAG: LangChain Full Walkthrough

`RAG`, `PINECONE`, `OPENAI`

In [37]:
# https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb

from tqdm.auto import tqdm
import os, time
from datasets import load_dataset
from pprint import pprint
import pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import langchain_pinecone

chat = ChatOpenAI()
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")


def augment_prompt(vectorstore: langchain_pinecone.Pinecone,
                   query: str, k: int = 3) -> str:
  # get top 3 results from knowledge base
  results = vectorstore.similarity_search(query, k=k)

  # get the text from the results
  source_knowledge = "\n".join([x.page_content for x in results])

  augmented_prompt = f"""You are a helpful assistant.

Using the contexts below, answer the query.
  
Contexts:
{source_knowledge}

Query: {query}"""

  return augmented_prompt

In [ ]:
"""
dataset = load_dataset(
  "jamescalam/llama-2-arxiv-papers-chunked",
  split="train"
)


data = dataset.to_pandas()
batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
  i_end = min(len(data), i + batch_size)
  # get batch of data
  batch = data.iloc[i:i_end]
  # generate unique ids for each chunk
  ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
  # get text to embed
  texts = [x['chunk'] for _, x in batch.iterrows()]
  # embed text
  embeds = embed_model.embed_documents(texts)
  # get metadata to store in Pinecone
  metadata = [
    {'text': x['chunk'],
    'source': x['source'],
    'title': x['title']} for i, x in batch.iterrows()
  ]
  # add to Pinecone
  index.upsert(vectors=zip(ids, embeds, metadata))

pprint(index.describe_index_stats())
"""

In [21]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
spec = pinecone.ServerlessSpec(cloud="gcp-starter", region="Iowa (us-central1)")

index_name = "chatbot"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
  pc.create_index(index_name, dimension=1536, metric="dotproduct", spec=spec)
  while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)


pprint(index.describe_index_stats())

text_field = "text"  # metadata field contains text chunk
vectorstore = langchain_pinecone.Pinecone(
  index, embed_model, text_field
)

{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}


In [38]:
query = "What is so special about Llama 2?"
prompt = augment_prompt(vectorstore=vectorstore, query=query, k=3)
res = chat.invoke(prompt)


# Tech With Tim


## ADVANCED Python AI Agent Tutorial - Using RAG

`RAG`, `AGENT`, `TOOLS`, `MULTILE DATA SOURCES`, `PROMPT INSTRUCTIONS`

### `tool_csv.py`

In [5]:
from llama_index.core import PromptTemplate
import pandas as pd
import os
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

instruction_str = """\
1. Convert the query to executable Python code using Pandas.
2. The final line of code should be a Python expression that can be called with the `eval()` function.
3. The code should represent a solution to the query.
4. PRINT ONLY THE EXPRESSION.
5. Do not quote the expression."""

template_pandas = """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}

Query: {query_str}

Expression:  """
prompt_template_pandas = PromptTemplate(template_pandas)

data_csv = pd.read_csv("./data/population.csv")
engine_query_csv = PandasQueryEngine(
  df=data_csv, verbose=True, instruction_str=instruction_str
)
engine_query_csv.update_prompts({"pandas_prompt": prompt_template_pandas})

tool_csv = QueryEngineTool(
    query_engine=engine_query_csv,
    metadata=ToolMetadata(
        name="population_data",
        description="this gives information about the world population and demographics",
    )
)
# pprint(engine_query_csv.query("what is the population of canada?"))

### `tool_note.py`

In [3]:
from llama_index.core.tools import FunctionTool

note_path = "./data/notes.txt"

def save_note(note: str, note_path: str):
  if not os.path.exists(note_path):
    open(note_path, "w")
  
  with open(note_path, "a") as f:
    f.writelines([f"{note}\n"])
    
  return "Note Saved"

tool_note = FunctionTool.from_defaults(
  fn=save_note,
  name="note_saver",
  description="this tool can save a text based note to a file for the user",
)

### `tool_pdf.py`

In [7]:
from llama_index.core import (StorageContext, VectorStoreIndex, 
                              load_index_from_storage)
from llama_index.readers.file import PDFReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata

reader_pdf = PDFReader()

def get_index(data, index_name):
  index = None
  
  if not os.path.exists(index_name):
    print(f"Building index '{index_name}'")
    index = VectorStoreIndex.from_documents(data, show_progress=True)
    index.storage_context.persist(persist_dir=index_name)
  else:
    print(f"Loading index '{index_name}'")
    index = load_index_from_storage(
      StorageContext.from_defaults(persist_dir=index_name)
    )
  
  return index

data_pdf = reader_pdf.load_data(file="./data/Canada.pdf") # docs
index_pdf = get_index(data_pdf, "index_pdf")
engine_query_pdf = index_pdf.as_query_engine()
tool_pdf = QueryEngineTool(
  query_engine=engine_query_pdf,
  metadata=ToolMetadata(
    name="canada_data",
    description="this gives detailed information about the country canada",
  )
)

Loading index 'index_pdf'


### `main.py`

In [9]:
from pprint import pprint
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

tools = [
  tool_note,
  tool_csv,
  tool_pdf,
]

context = """\
Purpose: The primary role of this agent is to assist users by providing accurate\
information about world population statistics and details about a country."""

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = ReActAgent.from_tools(tools=tools, llm=llm, verbose=True, context=context)


In [10]:

"""
Can you save a note for me with content: "I love you"
What is the population of Vatican City?
What is the population of India?
Tell me about the language token in Canada.
"""
while (prompt := input("Enter a prompt (q to quit)")) != "q":
  result = agent.query(prompt)
  print(result)

Thought: I can use the "canada_data" tool to get information about the language token in Canada.
Action: canada_data
Action Input: {'input': 'language'}
Observation: Canada is a linguistically diverse country with a significant portion of the population being able to speak both English and French. French is used as a language of instruction, in courts, and for government services in addition to English in some provinces. There are also Indigenous languages with official status in certain territories. Additionally, Canada is home to various sign languages, including American Sign Language (ASL) and Quebec Sign Language (LSQ).
Thought: I have enough information to answer the question.
Answer: In Canada, the language token is diverse. English and French are the official languages, with French being used in some provinces for government services, courts, and instruction. There are also Indigenous languages with official status in certain territories, as well as various sign languages such 

# Krish Naik


## Step-by-Step Guide to Building a RAG LLM App with LLamA2 and LLaMAindex
`RAG`, `LLAMA2`, `LLAMAINDEX`

In [5]:
!huggingface-cli login

In [ ]:
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import (VectorStoreIndex, SimpleDirectoryReader,
                              ServiceContext)
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
from llama_index.embeddings.langchain import LangchainEmbedding

documents = SimpleDirectoryReader("./data_small/Apple.pdf").load_data()

system_prompt = """\
You are a Q&A assistant. Your goal is to answer questions as accurately as\
possible based on the instructions and context provided.\
"""
template = """\
<|USER|>{query_str}<|ASSISTANT|>\
"""
query_wrapper_prompt = PromptTemplate(template)


llm = HuggingFaceLLM(
  context_window=4096,
  max_new_tokens=256,
  generate_kwargs={"temperature": 0.0, "do_sample": False},
  system_prompt=system_prompt,
  query_wrapper_prompt=query_wrapper_prompt,
  tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
  model_name="meta-llama/Llama-2-7b-chat-hf",
  device_map="auto",
  model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

service_context = ServiceContext.from_defaults(
  chunk_size=1024,
  llm=llm,
  embed_model=embed_model,
)

index = VectorStoreIndex.from_documents(
  documents=documents,
  service_context=service_context
)

query_engine = index.as_query_engine()

response = query_engine.query("what is attention is all you need?")

## End to end RAG LLM App Using Llamaindex and OpenAI- Indexing and Querying Multiple pdf's

`RAG`, `LLAMAINDEX`, `MULTIPLE FILES`

In [3]:
import os
from llama_index.core import (SimpleDirectoryReader, VectorStoreIndex,
                              StorageContext, load_index_from_storage)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


PERSIST_DIR = "./index"
INDEX_NAME = "husky_apple"
INDEX_DIR = f"{PERSIST_DIR}/{INDEX_NAME}"
if not os.path.exists(INDEX_DIR):
  print(f"Create index: '{INDEX_DIR}'")
  documents = SimpleDirectoryReader('./data_small').load_data()
  vectorstore_index = VectorStoreIndex.from_documents(
    documents=documents, 
    show_progress=True
  )
  vectorstore_index.storage_context.persist(persist_dir=INDEX_DIR)
else:
  print(f"Load index: '{INDEX_DIR}'")
  storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
  vectorstore_index = load_index_from_storage(storage_context)

retriever = VectorIndexRetriever(index=vectorstore_index, similarity_top_k=4)
post_processor = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(
  retriever=retriever,
  node_postprocessors=[post_processor],
)
response = query_engine.query("What is an apple?")
pprint(response)

Create index: './index/husky_apple'


/home/doantronghieu/anaconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2/2 [00:00<00:00,  3.04it/s]


# freeCodeCamp.org

## OpenAI, Langchain, Agents, Chroma
Development with Large Language Models Tutorial – OpenAI, Langchain, Agents, Chroma


# !